### Python imports

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import os
import time
import json
import boto3
import datetime
import requests
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
lambda_dir = os.path.join(root_dir, 'src', 'SeasonPredictor')

os.chdir(lambda_dir)
import lambda_function as season_lambda
os.chdir(root_dir)

import src.SeasonPredictor.aws4_requests

boto3.setup_default_session(profile_name='innovation-playground')
dynamo = boto3.client(service_name='dynamodb', region_name='eu-west-1')
ssm = boto3.client(service_name='ssm', region_name='eu-west-1')
parameter = ssm.get_parameter(Name='/adp/opta/feeds/auth/key', WithDecryption=True)
OUTLET_AUTH_KEY = parameter['Parameter']['Value']
IP_OR_DNS = ['api.performfeeds.com', '23.214.191.97', '35.176.250.248', '96.6.246.25'][1]

SEASON_UUIDS = {
    '2020/21 EPL': '8g80ydy1enmztpyanmadpofoq',
    '2020/21 LaLiga': '2web6ub1288xgby33z0vyc1uy',
    '2020/21 Serie A': '4b80uzt9gxak7d1vaa5jp17qi',
    '2020/21 Ligue 1': '5rr3izkmap6j5hfen757fq44q',
    '2020/21 Bundesliga': '3pgp7unogn1qfsg93jmi7x10q',
    '2020/21 Belgium Pro League': 'c4j3tbw4w4uze1v1ghjpvq5pm',
    '2020/21 EFL Championship': '1eeaaimnu0lb7c9y4djcya7f8',
    '2020/21 Eredivisie': 'bp1sjjiswd4t3nw86vf6yq7hm',
    '2020/21 Ligue 2': 'bgyywjjm7p7iyn18dutrdozfu',
    '2020/21 2. Bundesliga': '7u6i088r32wrl84442qxr0gh6',
    '2020/21 Serie B': '72faiinu2u54lfupciwz6nzmc',
    '2020/21 Segunda División': '28p3zfewe0zpuxxh1prqjv85w',
}

USE_LOCAL_DATA = True

In [ ]:
def visualize_league_standing_predictions(data):
    n_teams = len(data)
    default_font_size = 10 if n_teams <= 20 else 8
    current_points = [team_data['current']['points'] for team_data in data]
    team_order = np.flipud(np.argsort(current_points))

    fig = plt.figure(facecolor='w', figsize=(9, 5))

    y_tick_labels = []
    for ind_team, team_idx in enumerate(team_order):
        team_name = data[team_idx]['name']
        current_stats = data[team_idx]['current']

        # y_tick_labels.append(team_name)
        # y_tick_labels.append('{0:s} (G{1:d} W{2:d} D{3:d} L{4:d})'.format(team_name, current_stats['G'], current_stats['W'], current_stats['D'], current_stats['L']))
#         y_tick_labels.append('{0:s} ({1:d})'.format(team_name, current_stats['points']))
        y_tick_labels.append('{0:s} ({1:d}:{2:d})'.format(team_name, current_stats['points'], current_stats['G']))

        rank_probs = []
        for league_pos in range(n_teams):
            try:
                pos_prob = data[team_idx]['predicted']['rank'][str(league_pos + 1)]['p']
            except KeyError:
                pos_prob = 0
            rank_probs.append(pos_prob)
            plt.gca().add_patch(Rectangle((league_pos, ind_team), 1, 1, facecolor='r', alpha=pos_prob))
            if pos_prob > 0:
                if pos_prob == 1:
                    prob_txt = '1'
                    font_size = default_font_size
                else:
                    prob_txt = '{0:.2f}'.format(pos_prob)[1:]
                    font_size = default_font_size
                if prob_txt == '.00':
                    prob_txt = '{0:.3f}'.format(pos_prob)[1:]
                    font_size = default_font_size - 2
                if prob_txt == '.000':
                    prob_txt = '{0:.4f}'.format(pos_prob)[1:]
                    font_size = default_font_size - 4
                color_txt = 'r' if pos_prob < 0.65 else 'k'
                alpha_txt = np.sqrt(pos_prob) if pos_prob < 0.65 else 1
                plt.text(league_pos + 0.5, ind_team + 0.5, prob_txt, size=font_size, horizontalalignment='center', verticalalignment='center', color=color_txt, alpha=alpha_txt)

    margin = 0

    # Grid lines
    line_pos_list = {
        '2020/21 EPL': [4, 5, 17],
        '2020/21 LaLiga': [4, 6, 17],
        '2020/21 Serie A': [4, 6, 17],
        '2020/21 Ligue 1': [4, 6, 7, 17],
        '2020/21 Bundesliga': [4, 6, 15, 16],
        '2020/21 Belgium Pro League': [4, 8, 16, 17],
        '2020/21 EFL Championship': [2, 6, 21],
        '2020/21 Eredivisie': [1, 3, 7, 15, 16],
        '2020/21 Ligue 2': [2, 5, 17,18],
        '2020/21 2. Bundesliga': [2, 3, 15, 16],
        '2020/21 Serie B': [2, 8, 15, 17],
        '2020/21 Segunda División': [2, 6, 18],
    }[competition_picker.value]
    for line_pos in line_pos_list:
        plt.plot((0 - margin, n_teams + margin), (line_pos, line_pos), color='k', alpha=0.25)
        plt.plot((line_pos, line_pos), (0 - margin, n_teams + margin), color='k', alpha=0.25)

    plt.xlim(0 - margin, n_teams + margin)
    plt.ylim(n_teams + margin, 0 - margin)
    plt.xticks(np.arange(0, n_teams) + 0.5, np.arange(1, n_teams + 1).astype(str))
    plt.yticks(np.arange(0, n_teams) + 0.5, y_tick_labels)
    plt.xlabel('League position')

    ax = plt.gca().twiny()
    ax.set_xlim(0 - margin, n_teams + margin)
    ax.set_xticks(np.arange(0, n_teams) + 0.5)
    ax.set_xticklabels(np.arange(1, n_teams + 1).astype(str))

    plt.title('{0:s} end-of-season standing predictions\n'.format(competition_picker.value))
    plt.tight_layout()

In [ ]:
def update_league_plot(competition_name, n_runs):
    season_uuid = SEASON_UUIDS[competition_name]
    
    if USE_LOCAL_DATA:
        try:
            with open(os.path.join('notebooks', 'local_data_{}.json'.format(season_uuid)), 'r') as f:
                pred_data = json.load(f)
        except FileNotFoundError:
            print('Could not find simulation data locally for {0:s}'.format(competition_name))
            return
    else:    
        event_dict = {
            "tcal_ids": [season_uuid],
            "team_ids": "all",
            "prediction_types": ["league"],
            "n_runs": int(n_runs)
        }
        if len(what_if_results) > 0:
            event_dict['what_if'] = what_if_results

        status_indicator = widgets.HTML(value="Running simulations...")
        display(status_indicator)

        t0 = time.time()
        pred_response = season_lambda.lambda_handler(event_dict, None)
        pred_data = json.loads(pred_response['data'])[0][season_uuid]
        with open(os.path.join('notebooks', 'local_data_{}.json'.format(season_uuid)), 'w') as f:
            json.dump(pred_data, f)
        t1 = time.time()

        time_elapsed = t1 - t0
        time_mins = int(np.floor(time_elapsed / 60))
        time_seconds = time_elapsed % 60
        status_indicator.value = 'Simulation completed! Time elapsed: {0:s}{1:.2f} seconds'.format(
            '{0:d} minutes, '.format(time_mins) if time_mins > 0 else '', time_seconds
        )

    visualize_league_standing_predictions(pred_data['season_predictions'])
     
    global what_if_data
    what_if_data = pd.DataFrame(pred_data['game_list']).set_index('game_description', drop=False)
    for w in [what_if_picker, what_if_home_score, what_if_away_score, what_if_add_score_btn, what_if_reset_btn]:
        w.disabled = False
    what_if_picker.options = what_if_data['game_description'].to_list()
    what_if_picker.value = what_if_data['game_description'].values[0]
    
    return


def what_if_update(change):
    game_description = what_if_picker.value
    game_id = what_if_data.loc[game_description, 'game_id']
    home_score = int(what_if_home_score.value)
    away_score = int(what_if_away_score.value)
    txt = ''
    global what_if_results
    what_if_results[game_id] = {
        'description': game_description,
        'score': [home_score, away_score]
    }
    what_if_text.value = '<br>'.join(['{0:s}: {1:d}-{2:d}'.format(v['description'], v['score'][0], v['score'][1]) for k, v in what_if_results.items()])
#     for k, v in what_if_results.items():
#         txt += '{0:s}: {1:d}-{2:d}<br>'.format(k, v['score'][0], v['score'][1])


def what_if_reset(change):
    global what_if_results
    what_if_results = {}
    what_if_status_txt.value = ''
    what_if_text.value = ''
    
    
def update_game_status(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if what_if_data.loc[change['new'], 'game_status'] in ['Played', 'Awarded']:
            what_if_status_txt.value = 'Status: {0:s} ({1:d}-{2:d})'.format(
                *what_if_data.loc[change['new'], ['game_status', 'home_team_score', 'away_team_score']].to_list()
            )
        else:
            what_if_status_txt.value = 'Status: {0:s}'.format(
                what_if_data.loc[change['new'], 'game_status']
            )
    

competition_picker = widgets.Dropdown(
    options=list(SEASON_UUIDS.keys()),
    description='Competition'
)
n_runs_picker = widgets.FloatLogSlider(
    value=1000,
    base=10,
    min=2,
    max=5,
    step=1,
    description='# runs'
)

what_if_picker = widgets.Dropdown(
    options=[],
    description='What-if?',
    ensure_option=True,
    disabled=True,
    layout=widgets.Layout(width='50%')
)
what_if_status_txt = widgets.HTML(value="")
what_if_text = widgets.HTML(value="")
what_if_home_score = widgets.Dropdown(
    options=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
    value='0',
    description='Home score:',
    disabled=True,
    layout=widgets.Layout(width='15%')
)
what_if_away_score = widgets.Dropdown(
    options=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
    value='0',
    description='Away score:',
    disabled=True,
    layout=widgets.Layout(width='15%')
)
what_if_add_score_btn = widgets.Button(
    description='Add score',
    disabled=True,
)
what_if_reset_btn = widgets.Button(
    description='Reset',
    disabled=True,
)
    
_ = interact_manual(update_league_plot, competition_name=competition_picker, n_runs=n_runs_picker)

what_if_data = None
what_if_results = {}
what_if_picker.observe(update_game_status)
competition_picker.observe(what_if_reset)
what_if_add_score_btn.on_click(what_if_update)
what_if_reset_btn.on_click(what_if_reset)
display(widgets.VBox([
    widgets.HBox([what_if_picker, what_if_status_txt]),
    widgets.HBox([what_if_home_score, what_if_away_score, what_if_add_score_btn, what_if_reset_btn])
]))
display(what_if_text)

USE_LOCAL_DATA = False